# Getting user's location, current time and current weather
## Please run cell below to get your current location, time and weather condition

In [1]:
from ui.gui_requests_handler import get_current_location, get_current_date_time, get_current_weather

# Running GUI to get current user location
location = get_current_location()
print(f"Current location: ({location[0]}, {location[1]})")

# Getting current time
date_time = get_current_date_time()
print(f"Current data_time: {date_time}")

# Getting current weather
weather = get_current_weather(location)
print(f"Current weather: {weather}")

Current location: (59.850938, 30.319084)
Current data_time: 2021-10-27 01:14:33.635161
Current weather: 90


# Getting images

In [6]:
# Importing required modules
from database.database_client_sqlite import ImageDatabaseSQLiteClient

# Creating new database client instance
db_client = ImageDatabaseSQLiteClient()
images = db_client.get_images(location, date_time, weather, hour_delta=10, month_delta=5, weather_delta=100)

print(f"Found {len(images)} images for you")

SELECT PhotoID FROM Photos WHERE TRUE AND ((cast(strftime('%H', time(PhotoDateTime)) as INTEGER) >= 15) OR (cast(strftime('%H', time(PhotoDateTime)) as INTEGER) <= 11)) AND ((cast(strftime('%m', date(PhotoDateTime)) as INTEGER) >= 5) OR (cast(strftime('%m', date(PhotoDateTime)) as INTEGER) <= 3)) AND (LocationLatitude BETWEEN 59.350938 AND 60.350938) AND (LocationLongitude BETWEEN 29.819084 AND 30.819084) AND (Weather BETWEEN 0 AND 100)
Found 9 images for you


In [7]:
from ipyleaflet import Map, Marker, Icon
from ipywidgets import HTML, Image

center = (59.9451, 30.2169)

m = Map(center=center, zoom=10)

for image in images:
    marker = Marker(location=image.exif_info.gps_info.location(), draggable=False)
    image_path = '/'.join(image.image_path.split('/')[5:])
    palette_path = '/'.join(image.palette_path.split('/')[5:])
    marker.popup = HTML(f'<img src="{image_path}"></img>  <img src="{palette_path}"></img>')
    m.add_layer(marker)

m

Map(center=[59.9451, 30.2169], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…